# COPY INTO and MERGE Command

## COPY INTO Command
-   Incrementally loads data into Delta Lake tables from cloud Staorage
-   Supports Schema evolution
-   Supports wide range of file formats(CSV, JSON, Parquet, Delta)
-   Alternative to Auto Loader for batch ingestion

#### Create the table to copy the data into

In [0]:
create table if not exists demo.delta_lake.raw_stock_prices;

We have not added any columns to desmonstrate schema evolution with copy into command

#### Incrementally load new files into the table

In [0]:
copy into demo.delta_lake.raw_stock_prices
from 'abfss://demo@deacourseextdldev.dfs.core.windows.net/landing/stock_prices'
fileformat = json
format_options ('inferSchema' = 'true')
copy_options ('mergeSchema' = 'true')

In [0]:
select * from demo.delta_lake.raw_stock_prices

Add second file into ADLS Gen2 and rerun above commands

In [0]:
copy into demo.delta_lake.raw_stock_prices
from 'abfss://demo@deacourseextdldev.dfs.core.windows.net/landing/stock_prices'
fileformat = json
format_options ('inferSchema' = 'true')
copy_options ('mergeSchema' = 'true')

In [0]:
select *
from demo.delta_lake.raw_stock_prices;

# MERGE Statement
-   Used for Upserts (Insert/Update/Delete operations in a single statement)
-   Allows merging new data into a target table based on matching condition

### Create the Tabke to Merge the Data into

In [0]:
create table if not exists demo.delta_lake.stock_prices
(
  stock_id string,
  price double,
  trading_date date
);

### Merge the source data into the target table
  1. Insert new stocks received  
  2. Update price and trading_date if updates received
  3. Delete stocks which are de-listed from the exchange (status = 'DELISTED')

In [0]:
select * from demo.delta_lake.stock_prices;

In [0]:
merge into demo.delta_lake.stock_prices as target
using demo.delta_lake.raw_stock_prices as source 
   on target.stock_id = source.stock_id
when matched and source.status = 'ACTIVE'then 
  update set target.price = source.price, target.trading_date = source.trading_date
when matched and source.status = 'DELETED' then 
  delete
when not matched and source.status = 'ACTIVE' then
  insert (stock_id, price, trading_date) values (source.stock_id, source.price, source.trading_date);

In [0]:
select * from demo.delta_lake.stock_prices;

#### Let's upload 3rd file into ADLS Gen2

Once the 3rd file is loaded into ADLS Gen2 container, run copy into command to load the data into source first and then run merge command
to push the data into target table

In [0]:
delete from demo.delta_lake.raw_stock_prices;
-- Run this as the inital data from this table is already loaded or merged into target table and run copy into to load only new data
copy into demo.delta_lake.raw_stock_prices
from 'abfss://demo@deacourseextdldev.dfs.core.windows.net/landing/stock_prices'
fileformat = json
format_options ('inferSchema' = 'true')
copy_options ('mergeSchema' = 'true')

In [0]:
select * from demo.delta_lake.raw_stock_prices;

In [0]:
merge into demo.delta_lake.stock_prices as target
using demo.delta_lake.raw_stock_prices as source 
   on target.stock_id = source.stock_id
when matched and source.status = 'ACTIVE'then 
  update set target.price = source.price, target.trading_date = source.trading_date
when matched and source.status = 'DELETED' then  -- It should be 'DELISTED' instead of 'DELETED'
  delete
when not matched and source.status = 'ACTIVE' then
  insert (stock_id, price, trading_date) values (source.stock_id, source.price, source.trading_date);

In [0]:
select * from demo.delta_lake.stock_prices;